In [1]:
import requests
import pandas as pd
import json  
from pandas.io.json import json_normalize  
import psycopg2
import time

from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
# Genereating the timestamps
timestr = time.strftime("%Y%m%d-%H%M%S")


In [3]:
# Retrieving the location IDs (just 1-Fred's for now)
link = 'https://api.omnivore.io/1.0/locations/'

headers = {
    'Api-Key': '3a01cc5779ef4e3784658607fc188719',
}

response = requests.get(link, headers=headers)
s = response.text
j = json.loads(s)


In [4]:
# creating a list of location IDs
df = json_normalize(j['_embedded']['locations'])
location_ids = df['id'].tolist()

In [5]:
# retrieving tickets for location IDs
link_new = link+location_ids[0]+"/tickets/"

headers = {
    'Api-Key': '3a01cc5779ef4e3784658607fc188719',
}

response = requests.get(link_new, headers=headers)
s = response.text
j = json.loads(s)


In [6]:
# Saving the raw tickets to a json
fname = "data_"+timestr+".json"
with open(fname, 'w') as f:
        json.dump(j, f)
    



In [7]:
# normalizing the json to a dataframe (table)
df = json_normalize(j['_embedded']['tickets'])  

In [8]:
# retrieving items and categories from the ticket
item_name = []
item_category = []
for x in df['_embedded.items']:
    x = json_normalize(x)
    if x.empty:
        item_name.append("None")
        item_category.append("None")
    else:
        item_all = x['_embedded.menu_item.name']
        cat_j = x['_embedded.menu_item._embedded.menu_categories']
        items = ', '.join(item_all.tolist())
        cat_all = []
        for q in cat_j:
            p = json_normalize(q)
            c = p['name']
            c = set(c)
            l = ', '.join(list(c))
            cat_all.append(l)
            cat_all = list(set(cat_all))
        categories = ', '.join((cat_all))

        item_name.append(items)
        item_category.append(categories)

In [9]:
# Creating a table of relevant features
new_df = pd.DataFrame()
new_df['Ticket_ID'] = df['id']
new_df['Total'] = df['totals.total']/100
new_df['Subtotal'] = df['totals.sub_total']/100
new_df['Tax'] = df['totals.tax']/100
new_df['Tips'] = df['totals.tips']/100
new_df['Opened At'] = df['opened_at']
new_df['Closed At'] = df['closed_at']
new_df['Order Type'] = df['_embedded.order_type.name']
new_df['Item Name'] = item_name
new_df['Category Name'] = item_category
new_df['Closed At'] = new_df['Closed At'].fillna("None")
# new_df.set_index("Ticket_ID", inplace = True) 

opened_unix = new_df['Opened At'].tolist()
closed_unix = new_df['Closed At'].tolist()
opened_ts = []
closed_ts = []
for i in opened_unix:
    ts = int(i)
    opened_ts.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

for j in closed_unix:
    ts = int(i)
    closed_ts.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

opened_ts,closed_ts
new_df['Opened At'] = opened_ts
new_df['Closed At'] = closed_ts



In [19]:
# Connecting to POSTGRE
hostname = '127.0.0.1'
username = 'postgres'
password = 'S8Huu6xxCBBZ3ep'
database = 'postgres'

connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
cursor = connection.cursor()


In [11]:
# Checking for tickets already in the postgre
cursor.execute( "SELECT ticketid FROM OmnivoreData" )
existing_tickets = []
for tId in cursor.fetchall() :
    existing_tickets.append(str(tId[0]))
    
new_df=new_df[~new_df['Ticket_ID'].isin(existing_tickets)]


In [12]:
# Writing the new tickets to POstgre

if not new_df.empty:
    for Ticket_ID, df in new_df.groupby('Ticket_ID'):
        ticketId = str(df['Ticket_ID'].values[0])
        total = str(df['Total'].values[0])
        subtotal = str(df['Subtotal'].values[0])
        tax = str(df['Tax'].values[0])
        tips = str(df['Tips'].values[0])
        openedAt = str(df['Opened At'].values[0])
        closedAt = str(df['Closed At'].values[0])
        orderType = str(df['Order Type'].values[0])
        itemName = str(df['Item Name'].values[0])
        catName = str(df['Category Name'].values[0])



        postgres_insert_query =  "INSERT INTO OmnivoreData VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
        record_to_insert = (ticketId,total,subtotal,tax,openedAt,closedAt,orderType,itemName,catName,tips)
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
    
    print(str(timestr)+" => Records inserted")

else:
    print(str(timestr)+" => Database is up to date")
    
    
    


20200821-150738 => Records inserted


In [13]:
# retrieving the items and categories for normalization
s = response.text
j = json.loads(s)
df = json_normalize(j['_embedded']['tickets']) 

In [14]:
item_data = pd.DataFrame(columns = ['Ticket_Id', 'Opened_At','Closed_At','Item_Name', 'Categories', 'Price'])

for Ticket_Id, tickets in df.groupby('id'):

    for x in tickets['_embedded.items']:
        x = json_normalize(x)
        if not x.empty:
            i_list = x.name.tolist()
            i_pricelist = x.price.tolist()
            category_list = []
            for y in x['_embedded.menu_item._embedded.menu_categories']:
                y = json_normalize(y)
                z = y.name.tolist()
#                 print(z)
                for i in range(len(z)):
#                     print(i)
                    z[i] = z[i].strip()
                z = list(set(z))
                category_list.append(z)
#             print(i_list)
#             print(category_list)

            opened_unix = tickets['opened_at'].tolist()
            closed_unix = tickets['closed_at'].tolist()
            opened_ts = []
            closed_ts = []
            for i in opened_unix:
                ts = int(i)
                opened_ts.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

            for j in closed_unix:
                ts = int(i)
                closed_ts.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))
            
            for i in range(len(category_list)):
                cat = ', '.join(category_list[i])
                item_data.loc[len(item_data)] = [Ticket_Id, opened_ts[0], closed_ts[0], i_list[i], cat, i_pricelist[i]/100]
#     print("*********")
 
# item_data         


In [17]:
# Writing normalized item data

cursor.execute( "SELECT ticketid FROM OmnivoreItemData" )
existing_tickets = []
for tId in cursor.fetchall() :
    existing_tickets.append(str(tId[0]))
    
item_data = item_data[~item_data['Ticket_Id'].isin(existing_tickets)]

In [18]:
if not item_data.empty:
    for i,row in item_data.iterrows():
        postgres_insert_query =  "INSERT INTO OmnivoreItemData VALUES (%s,%s,%s,%s,%s,%s);"
        record_to_insert = (str(row['Ticket_Id']),str(row['Opened_At']),str(row['Closed_At']),str(row['Item_Name']),str(row['Categories']),str(row['Price']))
#         print(record_to_insert)
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()

    
    print(str(timestr)+" => Records inserted")

else:
    print(str(timestr)+" => Database is up to date")
    
    

20200821-150738 => Records inserted


In [ ]:
#Writing normalized category data

In [20]:
category_data = pd.DataFrame(columns = ['Ticket_Id', 'Opened_At','Closed_At','Item_Name', 'Categories', 'Price'])
for i,row in item_data.iterrows():
    cat_list = row['Categories'].split(", ")
    for j in cat_list:
        category_data.loc[len(category_data)] = [row['Ticket_Id'], row['Opened_At'],row['Closed_At'],row['Item_Name'], j, row['Price']]

In [21]:

cursor.execute( "SELECT ticketid FROM OmnivoreCategoryData" )
existing_tickets = []
for tId in cursor.fetchall() :
    existing_tickets.append(str(tId[0]))
    
category_data = category_data[~category_data['Ticket_Id'].isin(existing_tickets)]

In [22]:
if not category_data.empty:
    for i,row in category_data.iterrows():
        postgres_insert_query =  "INSERT INTO OmnivoreCategoryData VALUES (%s,%s,%s,%s,%s,%s);"
        record_to_insert = (str(row['Ticket_Id']),str(row['Opened_At']),str(row['Closed_At']),str(row['Item_Name']),str(row['Categories']),str(row['Price']))
#         print(record_to_insert)
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
    print(str(timestr)+" => Records inserted")

else:
    print(str(timestr)+" => Database is up to date")
    
    
    
cursor.close()
connection.close()

20200821-150738 => Records inserted
